<script type="text/javascript" src="http://cdn.mathjax.org/mathjax/latest/MathJax.js?config=default"></script>
# 模型的评估
分类器的各种评估度量，随机子抽样，KFold等一系列有助于评估模型的方法

## 分类器性能度量
包括准确率、召回率，精度，F1.

度量 | 公式   
-|-
准确率 | $\frac{TP+TN}{P+N}$
错误率 | $\frac{FP+FN}{P+N}$
召回率、真正例率 | $\frac{TP}{P}$
特殊性、真负例率 | $\frac{TN}{N}$
精度 | $\frac{TP}{TP+FP}$
F1值、精度和召回率的调和均值 | $\frac{2 \times precision \times recall}{precision+recall}$
$F_\beta$其中$\beta$ 是非负实数 | $\frac{(1+\beta^2)\times precision \times recall}{\beta^2 \times precision+recall}$

TP表示真正例、TN表示真假例、FP表示假正例、FN表示假假例，P表示正例，N表示假例。

混淆矩阵，可以用来评估ROC，TPR是正确地判断为阳性的比率，FPR是错误地判断为阳性之比率

$${\displaystyle TPR=TP/(TP+FN)}$$
$${\displaystyle FPR=FP/(FP+TN)}$$

将同一模型每个阈值 的 (FPR, TPR) 座标都画在ROC空间里，就成为特定模型的ROC曲线。

![](https://upload.wikimedia.org/wikipedia/commons/5/5c/ROCfig.PNG)

如图，从左到右，TPR，FPR的值

**还有其他几个方面可以用来评估分类器**

* 速度：计算的开销
* 鲁棒性：模型抗噪声的能力
* 可伸缩性：模型面对大规模的数据，能否有效地构造分类器
* 可解释性：决策树和分类规则的解释性更强，但是随着模型变得更加复杂，比如梯度提升树，或者神经网络，模型的可解释性就会更低。

## HoldOut和随机二次抽样 
* **HoldOut**：就是将整个数据分成两个子集，一个训练集，一个验证集。训练集占的比率随着数据量的增大而增大。比如只有10000条数据，就可以使用7500作为训练集，若是又100万，就可以使用95万作为训练集。**可用于时间序列和大数据集**
* **随机二次抽样**：从训练集中随机抽出多少多少的数据，分为两个子集。
![](images/HoldOut.png)

In [26]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import datasets

data = datasets.load_iris()
label = data['target']
train = data['data']
del data

In [38]:
print(train.shape)
# holdout
X_train,X_val,Y_train,Y_val = train_test_split(train,label,test_size = 0.1,random_state = 0, shuffle = False)
print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

(150, 4)
(135, 4)
(15, 4)
(135,)
(15,)


In [39]:
Y_val

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [40]:
print(train.shape)
# 随机二次抽样   区别就在于是否随机
X_train,X_val,Y_train,Y_val = train_test_split(train,label,test_size = 0.1,random_state = 0, shuffle = True)
print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

(150, 4)
(135, 4)
(15, 4)
(135,)
(15,)


In [41]:
Y_val

array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1])

## 交叉验证 又称KFlod
将初始数据随机地划分为$k$个不相交的子集，选择第$i$个子集作为验证集，让模型训练$k$次，结果取$k$次结果的平均。

$HoldOut$就是$KFold$的特殊情况。





In [43]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle = True, random_state = 0) # 确定折数
for train_index, test_index in kf.split(train):
    print("TRAIN:", train_index)
    print("TEST:", test_index)
    X_train, X_val = train[train_index], train[test_index]
    y_train, y_val = label[train_index], label[test_index]
    print()

##  自助法Bootstrap  对小数据集，效果很好
从给定训练数据中*有放回的均匀抽样*，即：每当选中一个元组，它等可能地又被提取出来训练，允许多次训练选中同一个数据。假设每个元组被选中地概率为$1/d$，那么没有被选择（作为测试集）的概率为$(1 - 1/d)$。需要选择$d$次，那么一个元组在$d$次训练集的挑选中，都没有被选中的概率为$(1 - 1/d)^d$，当d趋于无穷的时候，没被选中的概率为$e^{-1} = 0.368$。

即模型的总体准确率为：
$$ Acc(M) = \sum_{i=1}^{k}(0.632\times Acc(M_i)_{test_set} + 0.368\times Acc(M_i)_{train_set}) $$

（1） 采用重抽样技术从原始样本中抽取一定数量（自己给定）的样本，此过程允许重复抽样。 

（2） 根据抽出的样本计算给定的统计量T。 

（3） 重复上述N次（一般大于1000），得到N个统计量T。 

（4） 计算上述N个统计量T的样本方差，得到统计量的方差。

![](https://img-blog.csdn.net/20180329101221914?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L2l0ZXJhdGU3/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

举个例子：假设我们的蓝色点代表男生；黄色点代表女生，我们想知道他们的比例是否大体相当。那么我们采用bootstrap的步骤则是： 
1. 每次采样10个人，看男女比例。 
2. 重复上述过程10000次，把每次的男女比例求平均，代表最终的男女比例。

## 使用统计显著性检验来选择模型



假设已有两个训练好的模型$M_1$，$M_2$。已经进行$10$折交叉验证，得到了每个模型的平均误差率。尽管两个模型的错误率看上去不同，但是差别不很显著。如何选择模型？如果两者之间的差别是偶然的，那又怎么选择？

给定一个模型，在交叉验证中计算的每个**错误率**都可以看成一种概率分布不同的独立样本。一般，样本空间都服从$k-1$个自由度的t分布，其中$k = 10$。（两个分布的函数，一般来说都是不同的）。所使用的假设检验是$t$检验，或者研究$t$检验$(student's  test)$。

假设这两个模型相同，两者的平均误差率为0。如果能够拒绝原假设，则就可以断言两个模型之间的差是统计显著的。此时就选择错误率较低的模型，或者模型融合。

通过假设置信度，查表，来判断是否拒绝假设。

In [4]:
from scipy.stats import ttest_rel
s1 = [620.16,866.50,641.22,812.91,738.96,899.38,760.78,694.95,749.92,793.94]
s2 = [958.47,838.42,788.90,815.20,783.17,910.92,758.49,870.80,826.26,805.48]
print("Null Hypothesis:mean(s1) == mean(s2)，α=0.05")
ttest,pval = ttest_rel(s1,s2)
if pval < 0.05:
	print("Reject the Null Hypothesis.")
else:
	print("Accept the Null Hypothesis.")

Null Hypothesis:mean(s1) == mean(s2)，α=0.05
Accept the Null Hypothesis.
